In [1]:


import sparknlp
 
spark = sparknlp.start(gpu = True) # for GPU training >> sparknlp.start(gpu = True) # for Spark 2.3 =>> sparknlp.start(spark23 = True)
import pyspark.sql.functions as F
from sparknlp.annotator import *
from sparknlp.base import *

from sparknlp.common import *

import sparknlp
from sparknlp.pretrained import PretrainedPipeline
 
print("Spark NLP version", sparknlp.version())
 
print("Apache Spark version:", spark.version)



22/11/16 16:35:39 WARN Utils: Your hostname, Vader resolves to a loopback address: 127.0.1.1; using 192.168.1.94 instead (on interface enp5s0f0)
22/11/16 16:35:39 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
:: loading settings :: url = jar:file:/home/sjhjrok/anaconda3/envs/NLP/lib/python3.9/site-packages/pyspark/jars/ivy-2.5.0.jar!/org/apache/ivy/core/settings/ivysettings.xml


Ivy Default Cache set to: /home/sjhjrok/.ivy2/cache
The jars for the packages stored in: /home/sjhjrok/.ivy2/jars
com.johnsnowlabs.nlp#spark-nlp-gpu_2.12 added as a dependency
:: resolving dependencies :: org.apache.spark#spark-submit-parent-90d52822-93ca-4636-ba2c-e549564626d3;1.0
	confs: [default]
	found com.johnsnowlabs.nlp#spark-nlp-gpu_2.12;4.2.2 in central
	found com.typesafe#config;1.4.2 in central
	found org.rocksdb#rocksdbjni;6.29.5 in central
	found com.amazonaws#aws-java-sdk-bundle;1.11.828 in central
	found com.github.universal-automata#liblevenshtein;3.0.0 in central
	found com.google.code.findbugs#annotations;3.0.1 in central
	found net.jcip#jcip-annotations;1.0 in central
	found com.google.code.findbugs#jsr305;3.0.1 in central
	found com.google.protobuf#protobuf-java-util;3.0.0-beta-3 in central
	found com.google.protobuf#protobuf-java;3.0.0-beta-3 in central
	found com.google.code.gson#gson;2.3 in central
	found it.unimi.dsi#fastutil;7.0.12 in central
	found org.project

22/11/16 16:35:40 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


Spark NLP version 4.2.2
Apache Spark version: 3.3.1


In [ ]:


from sparknlp.training import CoNLL

training_data = CoNLL().readDataset(spark, './eng_custom.train')
testing_data= CoNLL().readDataset(spark, './eng_custom.testa')



In [ ]:
import pyspark.sql.functions as F

#training_data.select(F.explode(F.arrays_zip('token.result', 'pos.result',  'label.result')).alias("cols")) \
#.select(F.expr("cols['0']").alias("token"),
#        F.expr("cols['1']").alias("pos"),
#        F.expr("cols['2']").alias("ner_label")).show(truncate=50)


In [ ]:
!mkdir ner_logs

In [ ]:


# You can use any word embeddings you want (Glove, Elmo, Bert, custom etc.)
document_assembler = DocumentAssembler() \
    .setInputCol('text') \
    .setOutputCol('document')

tokenizer = Tokenizer() \
    .setInputCols(['document']) \
    .setOutputCol('token')
embeddings = WordEmbeddingsModel.pretrained('glove_100d')\
          .setInputCols(["document", "token"])\
          .setOutputCol("embeddings")

nerTagger = NerDLApproach()\
      .setInputCols(["sentence", "token", "embeddings"])\
      .setLabelColumn("label")\
      .setOutputCol("ner")\
      .setMaxEpochs(500)\
      .setLr(0.003)\
      .setBatchSize(32)\
      .setRandomSeed(0)\
      .setVerbose(1)\
      .setValidationSplit(0.2)\
      .setEvaluationLogExtended(True) \
      .setEnableOutputLogs(True)\
      .setIncludeConfidence(True)\
      .setOutputLogsPath('ner_logs') # if not set, logs will be written to ~/annotator_logs
 #    .setGraphFolder('graphs') >> put your graph file (pb) under this folder if you are using a custom graph generated thru 4.1 NerDL-Graph.ipynb notebook
 #    .setEnableMemoryOptimizer() >> if you have a limited memory and a large conll file, you can set this True to train batch by batch 
    
ner_converter = NerConverter() \
    .setInputCols(['document', 'token', 'ner']) \
    .setOutputCol('ner_chunk')

ner_pipeline = Pipeline(stages=[
      embeddings,
      nerTagger,
      ner_converter
 ])



In [ ]:
%%time
ner_model = ner_pipeline.fit(training_data)
ner_model.stages[1].write().overwrite().save('outputs/ner_wiki_glove100d_en')

In [ ]:

#predictions = ner_model.transform(testing_data) 
#predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#           .select(F.expr("cols['0']").alias("token"),
#                   F.expr("cols['1']").alias("ground_truth"),
#                   F.expr("cols['2']").alias("prediction")).show(truncate=False)

In [ ]:
from sklearn.metrics import classification_report
 
#preds_df = predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
#                      .select(F.expr("cols['0']").alias("token"),
#                              F.expr("cols['1']").alias("ground_truth"),
#                              F.expr("cols['2']").alias("prediction")).toPandas()
 
#print(classification_report(preds_df['ground_truth'], preds_df['prediction']))

In [ ]:
#import shutil

#shutil.make_archive("/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en", 'zip', "/home/sjhjrok/Documents/NLP/outputs/ner_wiki_glove100d_en")

In [ ]:
text_list = [
    """Navagate to kitchen"""
]


In [ ]:
loaded_ner_model = NerDLModel.load("outputs/ner_wiki_glove100d_en")\
   .setInputCols(["sentence", "token", "embeddings"])\
   .setOutputCol("ner")

load_ner_pipeline = Pipeline(stages=[
      document_assembler,
      tokenizer,
      embeddings,
      loaded_ner_model,
      ner_converter
 ])


In [ ]:
from sparknlp.base import LightPipeline

text = "navigate to kitchen"
prediction_data = spark.createDataFrame([[text]]).toDF("text")
prediction_model = load_ner_pipeline.fit(prediction_data)
light = LightPipeline(prediction_model)
results = light.annotate(text)
print(results)
print(results['ner'])

In [ ]:
light.annotate("Navigate to kitchen")


In [ ]:
text = input("Enter Testing Text\n")
while(text != "exit"):
    results = light.annotate(text)
    print(results['ner'])
    text = input("Enter New Text\n")

In [ ]:
#empty_df = spark.createDataFrame([['']]).toDF('text')
#pipeline_model = ner_pipeline.fit(empty_df)
#df = spark.createDataFrame(pd.DataFrame({'text': text_list}))
#result = pipeline_model.transform(df)

In [ ]:




#from sparknlp_display import NerVisualizer



#result = predictions

#NerVisualizer().display(
#    result = result.collect()[0],
#    label_col = 'ner_chunk',
#    document_col = 'document'
#)




